In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df = pd.read_csv("titanic.csv")
df.head()
df.shape


(891, 12)

In [2]:
# df.info()
# df.describe()
df.describe()
df.describe(include=object)

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Hirvonen, Miss. Hildur E",male,347082,B96 B98,S
freq,1,577,7,4,644


In [3]:
tm = df.groupby("Survived").mean()
tm
v1 = tm.iloc[0,:]
v2 = tm.iloc[1,:]
abs((v2-v1)/(v1+v2))

PassengerId    0.002971
Pclass         0.129755
Age            0.038706
SibSp          0.077914
Parch          0.170176
Fare           0.372661
dtype: float64

In [4]:
print(df["Name"].head())

def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'
def title_map(title):
    if title in ['Mr']:
        return 1
    elif title in ['Master']:
        return 3
    elif title in ['Ms','Mlle','Miss']:
        return 4
    elif title in ['Mme','Mrs']:
        return 5
    else:
        return 2
    
df['title'] = df['Name'].apply(get_title).apply(title_map)   

df = df.drop(["PassengerId", "Name", "Ticket"], axis="columns")
df.info()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
title       891 non-null int64
dtypes: float64(2), int64(5), object(3)
memory usage: 69.7+ KB


In [5]:
df["Sex"] = df["Sex"].replace(["male", "female"], [0, 1])
df["Cabin"] = df["Cabin"].isna()
df = pd.get_dummies(df)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null int64
Age           714 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Cabin         891 non-null bool
title         891 non-null int64
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: bool(1), float64(2), int64(6), uint8(3)
memory usage: 59.2 KB


In [6]:
tm = df.groupby("Survived").mean()
tm
v1 = tm.iloc[0,:]
v2 = tm.iloc[1,:]
abs((v2-v1)/(v1+v2))

Pclass        0.129755
Sex           0.643977
Age           0.038706
SibSp         0.077914
Parch         0.170176
Fare          0.372661
Cabin         0.185190
title         0.374197
Embarked_C    0.331221
Embarked_Q    0.012168
Embarked_S    0.101449
dtype: float64

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split


In [8]:
# print(df.groupby("title").Age.mean())
# df["Age"][df["title"]==3] = 7
# mage = df.Age.mean()
# df.Age = df.Age.fillna(mage)
# df = df.drop(["title"], axis="columns")

df["Age"][df["Age"].isna()] = df["Age"].mean()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null int64
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Cabin         891 non-null bool
title         891 non-null int64
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: bool(1), float64(2), int64(6), uint8(3)
memory usage: 59.2 KB


/home/gaurav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [9]:
mf = df['Fare'].mean()
df['Fare'] = df['Fare']>mf

# df.loc[ df['Fare'] <= 100, 'Fare'] = 0
# df.loc[ df['Fare'] > 100, 'Fare'] = 3

# # convert from float to int
df['Fare'] = df['Fare'].astype(int)


y = df["Survived"]
df.drop("Survived", axis="columns", inplace=True)
X = df


In [10]:
# model = LogisticRegression()
# model = RandomForestClassifier()

model = XGBClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
model.fit(X_train, y_train)

print(model.score(X_test,y_test))

from sklearn.metrics import confusion_matrix

yp = model.predict(X_test)
print("Sur", sum(yp!=0))
print("Not Sur", sum(yp==0))
cm = confusion_matrix(y_test, yp)
cm

0.8432835820895522
Sur 88
Not Sur 180


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[156,  18],
       [ 24,  70]])